# Video Decode Unit (VDU) Demo Example: DECODE -> FAKESINK 

# Introduction

Video Decode Unit (VDU) in ZynqMP SOC is capable of decoding AVC/HEVC compressed video streams.

This notebook example shows File playback usecase - where it reads compressed video and audio(optional) data from file or HTTP URL (e.g. youtube video), does decoding of video content using VDU, decoding of audio using software Gstreamer element and renders output on DP/HDMI display. 


# Implementation Details

<img src="pictures/block-diagram-decode-display.png" align="center" alt="Drawing" style="width: 600px; height: 200px"/>

### Board Setup 
1. Connect 4k DP/HDMI display to board.
2. Connect Ethernet cable. Check Internet connectivity. It is required for downloading compressed file from web-server.
3. Connect serial cable to monitor logs on serial console.
4. If Board is connected to private network, then export proxy settings in /home/root/.bashrc file as below,      
    - create/open a bashrc file using "vi ~/.bashrc" 
        - Insert below line to bashrc file
            - export http_proxy="< private network proxy address >"
            - export https_proxy="< private network proxy address >"
        - Save and close bashrc file.





In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Run the Demo 

In [2]:
from ipywidgets import interact
import ipywidgets as widgets
from common import common_vdu_demo_decode_display
import os
from ipywidgets import HBox, VBox, Text, Layout, HTML

### Insert Input file path Or Youtube URL

In [3]:
#input_path=widgets.Text(value='',
#    placeholder='Insert file path',
#    description=r'Input File:',
    #style={'description_width': 'initial'},
#    disabled=False)
#input_url=widgets.Text(value='',
#    placeholder='"Insert Youtube URL"',
#    description=r'<b>OR</b> Youtube URL (in quotes):',
#    style={'description_width': 'initial'},
#    layout=Layout(width='50%', height='30px'),
    #style={'flex container': 'cross-end'},
#    disabled=False)
#display(HBox([input_path, input_url]))
#display(HBox([input_url, input_path]))

import ipywidgets as widgets

class InputSection(widgets.HBox):
    def __init__(self):
        super().__init__()
        
        self.input_path = widgets.Text(value='',
            placeholder='Insert file path',
            description=r'Input File:',
            disabled=False)
        self.input_url = widgets.Text(value='',
            placeholder='"Insert Youtube URL"',
            description=r'<b>OR</b> Youtube URL (in quotes):',
            style={'description_width': 'initial'},
            layout=Layout(width='50%', height='30px'),
            disabled=False)
        
        self.children = [self.input_path, self.input_url]
        self.layout = widgets.Layout(border='')
    
    def get_input_path(self):
        return self.input_path.value
    
    def get_input_url(self):
        return self.input_url.value
    
# Create input section
input_section = InputSection()

# Display input section
display(input_section)

InputSection(children=(Text(value='', description='Input File:', placeholder='Insert file path'), Text(value='…

In [4]:
#proxy_url=widgets.Text(value='',
#    placeholder='(optional)',
#    description='Proxy URL:',
    #style={'description_width': 'initial'},
#    disabled=False)
#proxy_url
#HBox([proxy_url, optional])

class ProxySection(widgets.HBox):
    def __init__(self):
        super().__init__()
        
        self.proxy_url = widgets.Text(value='',
            placeholder='(optional)',
            description='Proxy URL:',
            disabled=False)
        
        self.children = [self.proxy_url]
        self.layout = widgets.Layout(border='')
    
    def get_proxy_url(self):
        return self.proxy_url.value
    
# Create proxy section
proxy_section = ProxySection()

# Display proxy section
display(proxy_section)

ProxySection(children=(Text(value='', description='Proxy URL:', placeholder='(optional)'),), layout=Layout(bor…

### Video

In [5]:
import ipywidgets as widgets

class VideoSection(widgets.HBox):
    def __init__(self, codec_options=['avc', 'hevc'], sink_options={'kmssink': ['DP', 'HDMI'], 'fakevideosink': ['none']}, decoder_options=['allegrodecIP0', 'allegrodecIP1', 'allegrodecIP2', 'allegrodecIP3']):
        
        # Initialize parent VBox
        super().__init__()
        
        
        # Initialize codec options
        self.codec_options = codec_options
        self.codec_type = widgets.RadioButtons(
            options=self.codec_options,
            description='Video Codec:',
            style={'description_width': 'auto', 'button_width': '10px'}
        )
        
        self.codec_type.observe(self._update_codec_type, names='value')
        self.selected_codec = self.codec_type.value
        
        
        # Initialize video sink options
        self.sink_options = sink_options
        self.sink_type = widgets.RadioButtons(
            options=sorted(self.sink_options.keys(), key=lambda k: len(self.sink_options[k]), reverse=True),
            value='fakevideosink',
            description='Video Sink:',
            style={'description_width': 'auto', 'button_width': '10px'}
        )
        
        init = self.sink_type.value
        self.display_type = widgets.RadioButtons(
            #options=self.sink_options[self.sink_type.value],
            options=self.sink_options[init], # Set default value to 'none'
            description='Display:',
            style={'description_width': 'auto', 'button_width': '10px'}
        )
        
        # Initialize stream options
        self.decoder_options = decoder_options
        self.decoder_instance = widgets.RadioButtons(
            options=self.decoder_options,
            description='Dec Instance:',
            style={'description_width': 'auto', 'button_width': '10px'},
        )
        #init1 = self.stream_type.value
        #self.stream_num = widgets.RadioButtons(
            #options=self.stream_options[self.stream_type.value],
         #   options=self.stream_options[init1],
          #  description='Instance:',
           # style={'description_width': 'auto', 'button_width': '10px'}
        #)
        
        # Define widget layout
        self.layout = widgets.Layout(border='')
        
        # Add widgets to VBox
        self.children = [self.codec_type, self.sink_type, self.display_type, self.decoder_instance]
        
        # Attach update functions
        self.sink_type.observe(self._update_display_options, names='value')
        #self.stream_type.observe(self._update_stream_options, names='value')
        
    def _update_display_options(self, change):
        self.display_type.options = self.sink_options[self.sink_type.value]
        
    #def _update_stream_options(self, change):
        #self.stream_num.options = self.stream_options[self.stream_type.value]
        
    #def get_codec_type(self):
       # return self.codec_type.value
        
    def _update_codec_type(self, change):
        print("Inside the update codec type: value of codec type is:", self.codec_type.value)
        self.selected_codec = self.codec_type.value
        print("Inside the update codec type: value of self.selected_codec is:", self.selected_codec)
        #self.selected_codec = change.new
        
    def get_codec_type(self):
        print("Inside the get_codec_type value of self.selected_codec is:", self.selected_codec)
        return self.codec_type.value
        
        
# Create first video section
video1 = VideoSection()
#video_sections = [video1]

# Display first video section
display(video1)





VideoSection(children=(RadioButtons(description='Video Codec:', options=('avc', 'hevc'), style=DescriptionStyl…

In [6]:
# Add a new video section
button = widgets.Button(description='Add more decoder', button_style='success')
#def add_video_section(button):
  #  video2 = VideoSection()
   # display(video2)

# Define a counter for the number of video sections
count = 1

# Initialize empty lists of input and video sections
input_sections = [input_section]
proxy_sections = [proxy_section]
video_sections = [video1]



# Display button section
#display(widgets.VBox([video_container, button_widget]))
# Define the add_video_section function with the counter
def add_video_section(button):
    global count
    name = f'video{count+1}'
    
    # Create new instances of InputSection and VideoSection
    input = InputSection()
    proxy = ProxySection()
    video = VideoSection()
    
    #video._dom_classes = [name]  # set class name to object name
    #display(video)
    count += 1
    
    # Add the new instances to the existing lists
    input_sections.append(input)
    proxy_sections.append(proxy)
    video_sections.append(video)
    
    #print("add_video_section: count: ", count)
    #print("add video_section: name:", name)
    
    display(input)
    display(proxy)
    display(video)
    
    # Remove the previous button
    button.layout.display = 'none'
    
    # Add a new button below the new video instance
    new_button = widgets.Button(description='Add more decoder', button_style='success')
    new_button.on_click(add_video_section)
    display(new_button)


button.on_click(add_video_section)
display(button)


Button(button_style='success', description='Add more decoder', style=ButtonStyle())

InputSection(children=(Text(value='', description='Input File:', placeholder='Insert file path'), Text(value='…

ProxySection(children=(Text(value='', description='Proxy URL:', placeholder='(optional)'),), layout=Layout(bor…

VideoSection(children=(RadioButtons(description='Video Codec:', options=('avc', 'hevc'), style=DescriptionStyl…

Button(button_style='success', description='Add more decoder', style=ButtonStyle())

### Advanced options:

In [7]:
entropy_buffers=widgets.Dropdown(
    options=['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15'],
    value='5',
    description='Entropy Buffers Nos:',
    style={'description_width': 'initial'},
    disabled=False,)
show_fps=widgets.Checkbox(
    value=False,
    description='show-fps',
    #style={'description_width': 'initial'},
    disabled=False)
loop=widgets.Checkbox(
    value=False,
    description='Repeat Play',
    #style={'description_width': 'initial'},
    disabled=False)
HBox([entropy_buffers, show_fps, loop])

In [8]:
from IPython.display import clear_output
from IPython.display import Javascript

def run_all(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))

def clear_op(event):
    clear_output(wait=True)
    return

button1 = widgets.Button(
    description='Clear Output',
    style= {'button_color':'lightgreen'},
    #style= {'button_color':'lightgreen', 'description_width': 'initial'},
    layout={'width': '300px'}
)
button2 = widgets.Button(
    description='',
    style= {'button_color':'white'},
    #style= {'button_color':'lightgreen', 'description_width': 'initial'},
    layout={'width': '83px'},
    disabled=True
)
button1.on_click(run_all)
button1.on_click(clear_op)

In [11]:
import subprocess 

# Create a video section
#video1 = VideoSection()


# Get the selected codec option of the video section
#codec_type = video1.get_codec_type()

# Create another video section
#video2 = VideoSection()

# Get the selected codec option of the video section
#codec_type1 = video2.get_codec_type()


def start_demo(event):

    # Assign fixed values to audio-related variables
    audio_codec = 'none'
    audio_sink = 'none'
    audio_output = ''
    
    #clear_output(wait=True)
    arg = [];
    arg1 = [];
    args2 = [];
    args3 = [];
    arg = common_vdu_demo_decode_display.cmd_line_args_generator(input_section.input_path.value, input_section.input_url.value, video1.codec_type.value, audio_codec, video1.display_type.value, show_fps.value, loop.value, entropy_buffers.value, proxy_section.proxy_url.value, audio_sink, audio_output, video1.sink_type.value, video1.decoder_instance.value);
    
    cmd1 = "sh vdu-demo-decode-display.sh {} 2>&1".format(arg)
    process1 = subprocess.Popen(cmd1, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
    

    # Invoke the script file with the appropriate command line arguments
    if count >= 2:
       
        arg1 = common_vdu_demo_decode_display.cmd_line_args_generator(input_sections[1].input_path.value, input_sections[1].input_url.value, video_sections[1].codec_type.value, audio_codec, video_sections[1].display_type.value, show_fps.value, loop.value, entropy_buffers.value, proxy_sections[1].proxy_url.value, audio_sink, audio_output, video_sections[1].sink_type.value, video_sections[1].decoder_instance.value);
        
        cmd2 = "sh vdu-demo-decode-display.sh {} 2>&1".format(arg1)
        process2 = subprocess.Popen(cmd2, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)


    if count >= 3:
        
        arg2 = common_vdu_demo_decode_display.cmd_line_args_generator(input_sections[2].input_path.value, input_sections[2].input_url.value, video_sections[2].codec_type.value, audio_codec, video_sections[2].display_type.value, show_fps.value, loop.value, entropy_buffers.value, proxy_sections[2].proxy_url.value, audio_sink, audio_output, video_sections[2].sink_type.value, video_sections[2].decoder_instance.value);
        
        cmd3 = "sh vdu-demo-decode-display.sh {} 2>&1".format(arg2)
        process3 = subprocess.Popen(cmd3, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
    if count >= 4:
        
        arg3 = common_vdu_demo_decode_display.cmd_line_args_generator(input_sections[3].input_path.value, input_sections[3].input_url.value, video_sections[3].codec_type.value, audio_codec, video_sections[3].display_type.value, show_fps.value, loop.value, entropy_buffers.value, proxy_sections[3].proxy_url.value, audio_sink, audio_output, video_sections[3].sink_type.value, video_sections[3].decoder_instance.value);
       
        cmd4 = "sh vdu-demo-decode-display.sh {} 2>&1".format(arg3)
        process4 = subprocess.Popen(cmd2, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
    
    
    print("Process 1 output:")
    for line in process1.stdout:
        print(line.strip())
        
    if count >= 2:
        print("Process 2 output:")
        for line in process2.stdout:
            print(line.strip())
    if count >= 3:
        print("Process 3 output:")
        for line in process3.stdout:
            print(line.strip())
    if count >= 4:
        print("Process 4 output:")
        for line in process4.stdout:
            print(line.strip())
    
    
    return

button = widgets.Button(
    description='click to start vdu-decode-display demo',
    style= {'button_color':'lightgreen'},
    #style= {'button_color':'lightgreen', 'description_width': 'initial'},
    layout={'width': '300px'}
)

button.on_click(start_demo)
HBox([button, button2, button1])

sh common_vdu_demo_decode_display.sh -i /home/petalinux/AIR_4Kp30_nv12.avc -c avc -o fakevideosink -z allegrodecIP0
sh common_vdu_demo_decode_display.sh -i /home/petalinux/AIR_4Kp30_nv12.avc -c avc -o fakevideosink -z allegrodecIP0
Process 1 output:
pidof: invalid option -- 'x'
BusyBox v1.35.0 () multi-call binary.

Usage: pidof [NAME]...

List PIDs of all processes with names that match NAMEs
gst-launch-1.0 filesrc location=/home/petalinux/AIR_4Kp30_nv12.avc ! h264parse ! omxh264dec device=/dev/allegroDecodeIP0 ! queue max-size-bytes=0 ! fakevideosink
Setting pipeline to PAUSED ...
Pipeline is PREROLLING ...
Redistribute latency...
Redistribute latency...
Pipeline is PREROLLED ...
Setting pipeline to PLAYING ...
Redistribute latency...
New clock: GstSystemClock


# References
[1] https://xilinx-wiki.atlassian.net/wiki/spaces/A/pages/18842546/Xilinx+Video+Codec+Unit

[2] https://www.xilinx.com/support.html#documentation (Refer to PG252)